In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import keras
from keras.layers import Dense, Convolution2D, Flatten, Activation, MaxPool2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils

import datetime

In [2]:
ds = pd.read_csv("/home/vasu/all_projects/CB/fashion-mnist_train.csv")
data = ds.values[:15000]

In [3]:
mask = data[:,0] < 5
print mask.sum()

7462


In [4]:
data_01 , data_02 = [],[]

for ix in range(data.shape[0]):
    if mask[ix] == True:
        data_01.append(data[ix])
    else:
        data_02.append(data[ix])
        
data_01 = np.array(data_01)
data_02 = np.array(data_02)

print data_01.shape,data_02.shape

(7462, 785) (7538, 785)


In [6]:
split = int(0.80*data_01.shape[0])

X_train = data_01[:split, 1:]/255.0
X_test = data_01[split:, 1:]/255.0

X_train = X_train.reshape((-1,28,28,1))
X_test = X_test.reshape((-1,28,28,1))

Y_train = np_utils.to_categorical(data_01[:split,0],num_classes=5)
Y_test = np_utils.to_categorical(data_01[split:,0], num_classes=5)

print X_train.shape, Y_train.shape
print X_test.shape, Y_test.shape

(5969, 28, 28, 1) (5969, 5)
(1493, 28, 28, 1) (1493, 5)


In [7]:
model = Sequential()

model.add(Convolution2D(64, (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Convolution2D(32, (3,3), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Convolution2D(16, (3,3), activation='relu'))
model.add(Convolution2D(8, (3,3), activation='relu'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
__________

In [10]:
start = datetime.datetime.now()

hist = model.fit(X_train,Y_train, epochs=20,shuffle=True,batch_size=128,validation_data=(X_test,Y_test))

print datetime.datetime.now() - start

Train on 5969 samples, validate on 1493 samples
Epoch 1/20
5969/5969 [==============================] - 10s - loss: 0.8964 - acc: 0.6473 - val_loss: 0.5058 - val_acc: 0.8031
Epoch 2/20
5969/5969 [==============================] - 8s - loss: 0.4933 - acc: 0.8119 - val_loss: 0.4578 - val_acc: 0.8305
Epoch 3/20
5969/5969 [==============================] - 7s - loss: 0.4273 - acc: 0.8365 - val_loss: 0.3958 - val_acc: 0.8500
Epoch 4/20
5969/5969 [==============================] - 7s - loss: 0.3771 - acc: 0.8599 - val_loss: 0.3800 - val_acc: 0.8674
Epoch 5/20
5969/5969 [==============================] - 8s - loss: 0.3502 - acc: 0.8707 - val_loss: 0.3525 - val_acc: 0.8747
Epoch 6/20
5969/5969 [==============================] - 7s - loss: 0.3056 - acc: 0.8859 - val_loss: 0.3338 - val_acc: 0.8754
Epoch 7/20
5969/5969 [==============================] - 7s - loss: 0.2838 - acc: 0.8981 - val_loss: 0.2984 - val_acc: 0.8895
Epoch 8/20
5969/5969 [==============================] - 7s - loss: 0.2709 - 

In [11]:
transfer_model = Sequential(model.layers[:-4])

for ix in transfer_model.layers:
    ix.trainable = False
    
transfer_model.add(Dense(5))
transfer_model.add(Activation('softmax'))

transfer_model.summary()
transfer_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 2565      
__________

In [13]:
split = int(0.80*data_02.shape[0])

X_train = data_02[:split, 1:]/255.0
X_test = data_02[split:, 1:]/255.0

X_train = X_train.reshape((-1,28,28,1))
X_test = X_test.reshape((-1,28,28,1))

Y_train = np_utils.to_categorical(data_02[:split,0]-5,num_classes=5)
Y_test = np_utils.to_categorical(data_02[split:,0]-5, num_classes=5)

print X_train.shape, Y_train.shape
print X_test.shape, Y_test.shape

(6030, 28, 28, 1) (6030, 5)
(1508, 28, 28, 1) (1508, 5)


In [15]:
start = datetime.datetime.now()

hist = transfer_model.fit(X_train,Y_train, epochs=20,shuffle=True,batch_size=128,validation_data=(X_test,Y_test))

print datetime.datetime.now() - start

Train on 6030 samples, validate on 1508 samples
Epoch 1/20
6030/6030 [==============================] - 3s - loss: 0.9001 - acc: 0.7172 - val_loss: 0.4237 - val_acc: 0.9058
Epoch 2/20
6030/6030 [==============================] - 3s - loss: 0.3432 - acc: 0.9146 - val_loss: 0.2939 - val_acc: 0.9125
Epoch 3/20
6030/6030 [==============================] - 3s - loss: 0.2660 - acc: 0.9250 - val_loss: 0.2514 - val_acc: 0.9231
Epoch 4/20
6030/6030 [==============================] - 3s - loss: 0.2316 - acc: 0.9320 - val_loss: 0.2255 - val_acc: 0.9297
Epoch 5/20
6030/6030 [==============================] - 3s - loss: 0.2116 - acc: 0.9355 - val_loss: 0.2091 - val_acc: 0.9357
Epoch 6/20
6030/6030 [==============================] - 3s - loss: 0.1964 - acc: 0.9383 - val_loss: 0.1987 - val_acc: 0.9370
Epoch 7/20
6030/6030 [==============================] - 3s - loss: 0.1851 - acc: 0.9408 - val_loss: 0.1888 - val_acc: 0.9390
Epoch 8/20
6030/6030 [==============================] - 3s - loss: 0.1788 - a